In [1]:
#예제 1.
def deco(func):
    def inner():
        print('running inner()')
    return inner

In [3]:
@deco
def target():
    print('running target()')
target()

running inner()


In [4]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
#예제 2.
registry = []
def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()
if __name__ == '__main__':
    main()


running register(<function f1 at 0x78abd242d900>)
running register(<function f2 at 0x78abefec5630>)
running main()
registry -> [<function f1 at 0x78abd242d900>, <function f2 at 0x78abefec5630>]
running f1()
running f2()
running f3()


In [6]:
#예제 3.
promos = []
def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    return order.total() * 0.05 if order.customer.fiidelity >=1000 else 0

@promotion
def bulk_item_promo(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)

In [7]:
#예제 4.
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [8]:
b=6
f1(3)

3
6


In [9]:
#예제 5.
b=6
def f2(a):
    print(a)
    print(b)
    b=9
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [10]:
#예제 6.
b=6
def f3(a):
    global b
    print(a)
    print(b)
    b=9
f3(3)

3
6


In [11]:
b

9

In [12]:
f3(3)

3
9


In [13]:
b=30
f3(3)

3
30


In [14]:
#예제 7.
class Averager():
    def __init__(self):
        self.series = []
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [17]:
avg = Averager()
avg(10)

10.0

In [18]:
avg(11)

10.5

In [20]:
avg(12)

11.0

In [21]:
#예제 8
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    return averager

In [22]:
avg = make_averager()
avg(10)

10.0

In [23]:
avg(11)

10.5

In [24]:
avg(12)

11.0

In [25]:
#예제 9.
avg.__code__.co_varnames

('new_value', 'total')

In [26]:
avg.__code__.co_freevars

('series',)

In [27]:
avg.__closure__

(<cell at 0x78abb5db71c0: list object at 0x78abb5a09d40>,)

In [28]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [29]:
#예제 10.
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total/count
    return averager

In [30]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [31]:
#예제 11.
def make_averager():
    count =0
    total =0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    return averager

In [32]:
avg = make_averager()
avg(10)

10.0

In [33]:
#예제 12.
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [35]:
#예제 13.
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)


if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12323800s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000086s] factorial(1) -> 1
[0.00005647s] factorial(2) -> 2
[0.00008640s] factorial(3) -> 6
[0.00011186s] factorial(4) -> 24
[0.00013870s] factorial(5) -> 120
[0.00016715s] factorial(6) -> 720
6! = 720


In [36]:
factorial.__name__

'clocked'

In [37]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)

        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked


In [38]:
#예제 14.

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12308073s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000072s] factorial(1) -> 1
[0.00005841s] factorial(2) -> 2
[0.00010180s] factorial(3) -> 6
[0.00012827s] factorial(4) -> 24
[0.00015306s] factorial(5) -> 120
[0.00017905s] factorial(6) -> 720
6! = 720


In [39]:
factorial.__name__

'factorial'